In [1]:
# Importing the libraries

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import math
import pandas as pd
import numpy as np

In [ ]:
# Initialize the Chrome webdriver
driver = webdriver.Chrome()
driver.maximize_window()
driver.get('https://www.linkedin.com/login')

# Wait for the sign-in page to load
time.sleep(2)


In [ ]:
# Find the username and password input fields and provide credentials

username = "*********************"  # Replace with your LinkedIn username
password = "**********"  # Replace with your LinkedIn password
username_field = driver.find_element(By.ID, "username")
username_field.send_keys(username)
password_field = driver.find_element(By.ID, "password")
password_field.send_keys(password)
# Click the sign-in button
sign_in_button = driver.find_element(By.XPATH, "//button[@aria-label='Sign in']")
sign_in_button.click()
time.sleep(20)
 

In [ ]:
# Search for jobs

search_input = driver.find_element(By.CLASS_NAME, "search-global-typeahead__input")
job_role = "Data Analyst"
search_input.send_keys(job_role)
search_input.send_keys(Keys.ENTER)
time.sleep(5)
button_element = driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div[2]/section/div/nav/div/ul/li[1]/button")
button_element.click() # click on Jobs Button
button_element = driver.find_element(By.XPATH, "/html/body/div[5]/div[4]/aside[1]/div[1]/header/div[3]/button[2]")
button_element.click() # Minimize messaging overlay
time.sleep(5)



In [ ]:
# Create empty lists to store details 

activelyrecruiting = []
titles = []
company = []
links = []
exprience_level = []
employee_count__domain = []
description = []
skills = []

In [2]:
#Find the last Page for the job search result

total_jobs_element = driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div[4]/div/div/main/div/div[2]/div[1]/header/div[1]/small/div/span").text
numeric_part = ""
for char in total_jobs_element:
    if char.isdigit():
        numeric_part += char
total_jobs = int(numeric_part)
max_pages = math.ceil(total_jobs / 25)
current_page = 1

In [ ]:
#Scraping the details

while current_page <= max_pages:

    job_container = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "scaffold-layout__list"))
    )
    
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "jobs-search-results__list-item")))

    # Find all job listing elements within the container
    job_listings = job_container.find_elements(By.CLASS_NAME, "jobs-search-results__list-item")

    for index, job_listing in enumerate(job_listings):

        # time.sleep(1)
            
        try:
            jobactivelyrecruiting = driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div[4]/div/div/main/div/div[2]/div[1]/div/ul/li[1]/div/div/div[2]/div").text
            activelyrecruiting.append(jobactivelyrecruiting)
        except:
            activelyrecruiting.append(None)
        
        job_listing.click()
        time.sleep(2)
        click_counter = 1
        driver.switch_to.window(driver.window_handles[-1])
        driver.implicitly_wait(3)

        try:
            job_title_element = driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div[4]/div/div/main/div/div[2]/div[2]/div/div[2]/div/div[1]/div/div[1]/div/div[1]/div[1]/div[1]/h2").text
            titles.append(job_title_element)
        except NoSuchElementException:
            titles.append(None)
                                                
        try:
            job_link = driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div[4]/div/div/main/div/div[2]/div[2]/div/div[2]/div/div[1]/div/div[1]/div/div[1]/div[1]/div[1]/h2/a")
            links.append(job_link.get_attribute("href"))
        except:
            links.append(None)

        try:
            jobcompany = driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div[4]/div/div/main/div/div[2]/div[2]/div/div[2]/div/div[1]/div/div[1]/div/div[1]/div[1]/div[2]/div").text
            company.append(jobcompany)
        except:
            company.append(None)
            
        try:
            jobexprience_level = driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div[4]/div/div/main/div/div[2]/div[2]/div/div[2]/div/div[1]/div/div[1]/div/div[1]/div[1]/div[3]/ul/li[1]").text
            exprience_level.append(jobexprience_level)
        except:
            exprience_level.append(None)

        try:
            jobemployee_count__domain = driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div[4]/div/div/main/div/div[2]/div[2]/div/div[2]/div/div[1]/div/div[1]/div/div[1]/div[1]/div[3]/ul/li[2]/span").text
            employee_count__domain.append(jobemployee_count__domain)
        except:
            employee_count__domain.append(None)

        scroll_element = driver.find_element(By.CLASS_NAME, "jobs-search__job-details--wrapper")
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll_element) 
        
        try:
            job_description_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "/html/body/div[5]/div[3]/div[4]/div/div/main/div/div[2]/div[2]/div/div[2]/div/div[1]/div/div[2]/article/div/div[1]"))
            )
            job_description = job_description_element.text
            description.append(job_description)
        except:
            description.append(None)

        try:
            skill_elements = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.app-aware-link.job-details-how-you-match__skills-item-subtitle.t-14.overflow-hidden"))
            )
            skill_list = [skill.text.strip() for skill in skill_elements]
            skills.append(skill_list)
        except:
            skills.append(None)

        driver.switch_to.window(driver.window_handles[0])
        next_index = index + 1
        if next_index < len(job_listings):
            next_job_listing = job_listings[next_index]
            next_job_listing.click()
            driver.switch_to.window(driver.window_handles[-1])
        
        click_counter += 1
        # Check if it's time to scroll
        if click_counter % 2 == 0:
            # Scroll to the bottom of the page
            driver.execute_script("arguments[0].scrollIntoView();", job_container)

    next_button_xpath = f"//button[@aria-label='Page {current_page + 1}']"
    next_button = driver.find_element(By.XPATH, next_button_xpath)
    next_button.click()
    time.sleep(2)
    current_page += 1

# Close the WebDriver
driver.quit()

In [4]:
# Make a csv file for the scraped data

data = {
    'Activelyrecruiting' : activelyrecruiting,
    'Title': titles,
    'Company': company, # location of job included
    'Link': links,
    'Experience Level': exprience_level,
    'Employee Count/Domain': employee_count__domain,
    'Description': description,
    'Skills': skills
}

df = pd.DataFrame(data)
filename = "AnalystJobs.csv"
df.to_csv(filename, index=False)